In [3]:
import pandas as pd 
import numpy as np
import seaborn as sns
import cufflinks as cf
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
import re 
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
cf.go_offline();
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from textblob import TextBlob
from wordcloud import WordCloud
from langdetect import detect
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kshit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kshit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kshit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
data = pd.read_excel('C:/Users/kshit/Desktop/NLP Project/Reviews/amazon_reviews_opn5g.xlsx')
data

,product,profile_name,review_title,review_rating,review_text
0,OnePlus Nord 5G,Nikhil,*Read before you buy!!*,5.0 out of 5 stars,"\n Yea..pre-ordered on 28 July, got it on 4 A..."
1,OnePlus Nord 5G,Amit,Near to mid range Perfection,5.0 out of 5 stars,"\n Got it delivered yesterday , used for abou..."
2,OnePlus Nord 5G,aishwarya,Great price!,5.0 out of 5 stars,\n An amazing phone!
3,OnePlus Nord 5G,vasu a.,Beast in OnePlus.,5.0 out of 5 stars,\n Brilliant..
4,OnePlus Nord 5G,Amazon Customer,Changed to Nord from 6t,5.0 out of 5 stars,\n I was skeptical about changing from One pl...
...,...,...,...,...,...
9464,OnePlus Nord 5G,DHIRAJ KUMAR,Very bad quality,2.0 out of 5 stars,\n Very bad\n
9465,OnePlus Nord 5G,Farhan Ali,Descent,2.0 out of 5 stars,\n Descent\n
9466,OnePlus Nord 5G,Aftab N,waste,2.0 out of 5 stars,\n West m\n
9467,OnePlus Nord 5G,sukhmander singh,Oj,2.0 out of 5 stars,\n Ok ok\n


Pre-processing Steps :
1. Fixing contractions 
2. Removing punctuations 
3. Removing numbers
4. Converting to lowercase
5. Spell check using TextBlob
6. Tokenization 
7. Lemmatization 
8. Removing stop words
9. Joining together 

In [5]:
def clean_text(text):
    cleaned_text = contractions.fix(text)  # Fix contractions
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)  # Remove punctuation
    cleaned_text = re.sub(r'\d', '', cleaned_text)  # Remove digits
    cleaned_text = cleaned_text.lower()  # Convert to lowercase
    
    # Spell checking using TextBlob
    blob = TextBlob(cleaned_text)
    cleaned_text = str(blob.correct())
    
    return cleaned_text

# Apply the cleaning function to the 'review_text' column
data['s1'] = data['review_text'].apply(clean_text)
data['s1']

#13mins 


0       \n  yeapreordered on  july got it on  august p...
1       \n  got it delivered yesterday  used for about...
2                                    \n  an amazing phone
3                                           \n  brilliant
4       \n  i was skeptically about changing from one ...
                              ...                        
9464                                       \n  very bad\n
9465                                        \n  descent\n
9466                                         \n  west m\n
9467                                          \n  ok ok\n
9468                                          \n  medal\n
Name: s1, Length: 9469, dtype: object

In [6]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)  # Tokenize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]  # Remove stop words
    return ' '.join(filtered_tokens)

data['processed_reviews'] = data['s1'].apply(preprocess)
data['processed_reviews']

tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features
tfidf_matrix = tfidf_vectorizer.fit_transform(data['processed_reviews'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


In [7]:
data

,product,profile_name,review_title,review_rating,review_text,s1,processed_reviews
0,OnePlus Nord 5G,Nikhil,*Read before you buy!!*,5.0 out of 5 stars,"\n Yea..pre-ordered on 28 July, got it on 4 A...",\n yeapreordered on july got it on august p...,yeapreordered july got august package wa nice ...
1,OnePlus Nord 5G,Amit,Near to mid range Perfection,5.0 out of 5 stars,"\n Got it delivered yesterday , used for abou...",\n got it delivered yesterday used for about...,got delivered yesterday used hour tell first m...
2,OnePlus Nord 5G,aishwarya,Great price!,5.0 out of 5 stars,\n An amazing phone!,\n an amazing phone,amazing phone
3,OnePlus Nord 5G,vasu a.,Beast in OnePlus.,5.0 out of 5 stars,\n Brilliant..,\n brilliant,brilliant
4,OnePlus Nord 5G,Amazon Customer,Changed to Nord from 6t,5.0 out of 5 stars,\n I was skeptical about changing from One pl...,\n i was skeptically about changing from one ...,wa skeptically changing one plus word still pr...
...,...,...,...,...,...,...,...
9464,OnePlus Nord 5G,DHIRAJ KUMAR,Very bad quality,2.0 out of 5 stars,\n Very bad\n,\n very bad\n,bad
9465,OnePlus Nord 5G,Farhan Ali,Descent,2.0 out of 5 stars,\n Descent\n,\n descent\n,descent
9466,OnePlus Nord 5G,Aftab N,waste,2.0 out of 5 stars,\n West m\n,\n west m\n,west
9467,OnePlus Nord 5G,sukhmander singh,Oj,2.0 out of 5 stars,\n Ok ok\n,\n ok ok\n,ok ok


In [8]:
data.to_excel('processed_reviews.xlsx', index=False)
tfidf_df.to_excel('processed_reviews_tfidf.xlsx', index=False)


In [ ]:
columns_to_export = ['profile_name', 'processed_reviews'] # Specify the columns you want to export
selected_columns_df = data[columns_to_export] # Create a new DataFrame with only the selected columns
excel_filename = 'processed_reviews_select.xlsx'
selected_columns_df.to_excel(excel_filename, index=False)